In [56]:
import pandas as pd
from hs_gimme.db_facade.db_facade_factory import get_mongo_client_db
from collections import Counter, defaultdict
from datetime import datetime

In [59]:
mongo = get_mongo_client_db('production', 'jamaica')

In [ ]:
addresses = Counter([r['country'] for r in mongo.req.find({}, ['country']).limit(1000)])

In [ ]:
addresses

In [ ]:
nyc = [a for a in addresses.keys() if 'New York' in a]
nyc

In [5]:
q = {'job_create_date': {'$gte': datetime(2022, 1, 1), '$lte': datetime(2023, 6, 30)}, 'country': 'United States'}

In [6]:
reqs = list(mongo.req.find(q))
len(reqs)

2023-07-30T14:10:47.009279Z [warning  ] SlowLog                        command_name=getMore connection_id=mongodb-3c.prod.omcomcom.com:27029 duration_ms__numeric=41602.202 duration_threshold__numeric=10000 environment=local file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/slow_logs_utils.py function_name=write_slow_log hostname=Dima-Shulga-MacBook-Pro line_number=11 module=hs_gimme.db_facade.slow_logs_utils pid=42904 request_id=78098867 source=PyMongo stack_snippet=
  File "/var/folders/_p/8_m91crj199_rd7sndkxncc80000gp/T/ipykernel_42904/4174422903.py", line 1, in <module>
    reqs = list(mongo.req.find(q))



29795

In [7]:
min(r['job_create_date'] for r in reqs) 

datetime.datetime(2022, 1, 3, 11, 0, 41)

In [8]:
req_ids = [r['_id'] for r in reqs]

In [9]:
apps_p = ['ats_application.eeo', 
          'ats_application.integration_data.final_round_score_sent_to_ats',
          'grade_data.explainable_score_data.explain',
         'req_id']

In [10]:
apps = list(mongo.application.find({'req_id': {'$in': req_ids}}, apps_p))
len(apps)

2023-07-30T14:21:21.882580Z [warning  ] SlowLog                        command_name=getMore connection_id=mongodb-3c.prod.omcomcom.com:27029 duration_ms__numeric=13731.134 duration_threshold__numeric=10000 environment=local file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/slow_logs_utils.py function_name=write_slow_log hostname=Dima-Shulga-MacBook-Pro line_number=11 module=hs_gimme.db_facade.slow_logs_utils pid=42904 request_id=947609847 source=PyMongo stack_snippet=
  File "/var/folders/_p/8_m91crj199_rd7sndkxncc80000gp/T/ipykernel_42904/2093352448.py", line 1, in <module>
    apps = list(mongo.application.find({'req_id': {'$in': req_ids}}, apps_p))

2023-07-30T14:23:49.839508Z [warning  ] SlowLog                        command_name=getMore connection_id=mongodb-3c.prod.omcomcom.com:27029 duration_ms__numeric=11634.742 duration_threshold__numeric=10000 environment=local file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/slow_logs_utils.py function_name=write_slow_log h

1030702

In [11]:
genders = set(a['ats_application'].get('eeo', {}).get('gender', 'Missing') for a in apps)
genders

{'Decline to Identify', 'Female', 'Male', 'Missing'}

In [12]:
races = set(a['ats_application'].get('eeo', {}).get('race', 'Missing') for a in apps)
races

{'Asian',
 'Black or African American',
 'Decline to Identify',
 'Hispanic or Latino',
 'Missing',
 'Native American',
 'Native Hawaiian or Pacific Islander',
 'Two or more races',
 'White'}

In [13]:
req_to_apps = defaultdict(list)
for a in apps:
    req_to_apps[a['req_id']].append(a)


In [14]:
int_to_grade= {
                1: 'D',
                2: 'C',
                3: 'B',
                4: 'A'
            }

In [41]:
data = []
for i, req in enumerate(reqs):
    for j, app in enumerate(req_to_apps[req['_id']]):
        d = { 
            'id': 'ANONYMOUS{:04d}{:04d}'.format(i, j),
            'app_id': app['_id'],
            'candidate_id': app['_id'].split('_')[1].replace('JNJ', ''),
            'req_id': req['_id'].replace('JNJ', '')
        }
        
        for f in ['address',
                  'country',
                  'job_level',
                  'job_education', 
                  'job_department', 
                  'job_education', 
                  'job_function', 
                  'external_status',
                  'job_create_date',
                  'top_category',
                  'sub_category',
                  'min_years_of_relevant_experience']:
            d[f] = req.get(f)
            
        
        gender = app['ats_application'].get('eeo', {}).get('gender', 'Missing')
        d['gender'] = gender
        is_gender_na = gender in {'Missing', 'Decline to Identify'}
        for g in {'Male', 'Female'}:
            if is_gender_na:
                d[g] = 'Missing'
            else:
                d[g] = int(gender == g)
                
                
        race = app['ats_application'].get('eeo', {}).get('race', 'Missing')
        d['race'] = race
        is_race_na = race in {'Missing', 'Decline to Identify'}
        for r in {'Asian',
                 'Black or African American',
                 'Hispanic or Latino',
                 'Native American',
                 'Native Hawaiian or Pacific Islander',
                 'Two or more races',
                 'White'}:
            if is_race_na:
                d[r] = 'Missing'
            else:
                d[r] = int(race == r)
        
        
        if 'grade_data' in app and 'explainable_score_data' in app['grade_data']:
            d['int_grade'] = int(app['grade_data']['explainable_score_data']['explain']['raw_grade_without_rnd'])
            
            d['grade'] = int_to_grade[d['int_grade']]
              
            if d.get('grade') in {'A', 'B', 'C', 'D'}:
                data.append(d)
df = pd.DataFrame(data)

df.shape

(782098, 27)

In [42]:
# mask = (df['meets_qualifications'] == df['is_priority'])

# sample = df[mask].groupby(['gender', 'is_priority']).apply(lambda x: x.sample(min(50, len(x)))).reset_index(drop=True)
# sample = sample.replace('Missing', '')
# len(sample)
#mask = lambda x: x['address'] == 'New York, New York, United States'

#mask = lambda x: x['job_create_date'] > datetime(2022, 1, 1)
sample = df.replace('Missing', '')
len(sample)

782098

In [43]:
file_name = 'jul_30'

In [44]:
cols = ['candidate_id', 'req_id', 'job_level', 'job_department', 'job_function',
         'Male', 'Female',
       'Native Hawaiian or Pacific Islander', 'Black or African American',
       'Native American', 'White', 'Asian', 'Hispanic or Latino',
       'Two or more races',  'grade']

In [45]:

sample.to_csv(f'~/Downloads/JnJ_data_{file_name}.csv', index=False)

In [46]:
sample['race_disclose'] = sample['race'].apply(lambda x: 'Disclose' if x not in {'Decline to Identify', 'Missing'} else x)
sample['gender_disclose'] = sample['gender'].apply(lambda x: 'Disclose' if x not in {'Decline to Identify', 'Missing'} else x)

In [47]:
writer = pd.ExcelWriter(f'~/Downloads/JnJ_data_{file_name}.xlsx', engine="xlsxwriter")

In [48]:
f'~/Downloads/JnJ_data_{file_name}'

'~/Downloads/JnJ_data_jul_30'

In [52]:
min_date = min(sample['job_create_date']) 
max_date = max(sample['job_create_date']) 

summary = pd.Series({
    'Date': datetime.now(),
    'Reqs Locations': 'United States',
    'Reqs External Status': list(sample['external_status'].unique()),
    'Total number of Candidates': len(sample),
    'Total Number of reqs': sample['req_id'].nunique(),
    'Dates Range': f'{min_date} - {max_date}'
}).reset_index()


summary.to_excel(writer, sheet_name="Summary", index=False)
summary

,index,0
0,Date,2023-07-30 17:55:47.131872
1,Reqs Locations,United States
2,Reqs External Status,"[Filled, Canceled, On Hold, Open, Draft]"
3,Total number of Candidates,782098
4,Total Number of reqs,21460
5,Dates Range,2022-01-03 11:00:41 - 2023-06-29 22:00:48


In [53]:
sample[cols].to_excel(writer, sheet_name="Sample", index=False)

In [54]:
disclose = pd.DataFrame( {'Race': sample['race_disclose'].value_counts(normalize=False), 
               'Gender': sample['gender_disclose'].value_counts(normalize=False)})
disclose.to_excel(writer, sheet_name="% Disclose")
disclose

,Race,Gender
Disclose,750213,766785
Decline to Identify,31885,15313


In [55]:
writer.close()